In [1]:
import os

In [2]:

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:

import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import cityscapes_labels
import utils
from generator import *

from models import *
from keras import losses, metrics, optimizers
from metrics import precision, dice_coef


Using TensorFlow backend.


In [4]:
!nvidia-smi

Sun Apr 22 11:48:57 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   57C    P2    35W / 140W |   5939MiB /  6072MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [8]:
#target_height, target_width = 360, 480
#target_height, target_width = 360, 648
target_size = 256, 512

import config


labels = cityscapes_labels.labels
n_classes = len(labels)
batch_size = 2
epochs = 10

dataset_path = config.data_path()

# segnet
#model = SegNetWarp(target_size, n_classes)
model = ICNetWarp1(target_size, n_classes)

#model.k.load_weights('checkpoint/cityscapes/SegNetWarp/debug/warp_3_save_epoch.h5')
model.k.load_weights('/home/mlyko/weights/city/rel/ICNetWarp1/2207:43b16_lr=0.000900_dec=0.050000.h5')

model.summary()

model.compile()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data_new (InputLayer)           (None, 256, 512, 3)  0                                            
__________________________________________________________________________________________________
data_sub2 (Lambda)              (None, 128, 256, 3)  0           data_new[0][0]                   
__________________________________________________________________________________________________
conv1_1_3x3_s2 (Conv2D)         (None, 64, 128, 32)  896         data_sub2[0][0]                  
__________________________________________________________________________________________________
conv1_1_3x3_s2_bn (BatchNormali (None, 64, 128, 32)  128         conv1_1_3x3_s2[0][0]             
__________________________________________________________________________________________________
conv1_2_3x

In [7]:
!ls /home/mlyko/weights/city/rel/ICNetWarp1/

2207:43b16_lr=0.000900_dec=0.050000.h5
diff_bwb16_lr=0.000100_dec=0.005000.h5
lc1.0+0.99b16_lr=0.001000_dec=0.009000.h5
minmax_lc1-1.0+1.0b16_lr=0.002100_dec=0.003000_150_finished.h5
minmax_lc1-1.0+1.0b16_lr=0.002100_dec=0.003000.h5
opening_concatb16_lr=0.000900_dec=0.100000.h5
opening_concatb16_lr=0.002100_dec=0.003000_150_finished.h5
opening_concatb16_lr=0.002100_dec=0.003000.h5


In [9]:
# test visualize
import keras.backend as K

def layer_to_visualize(img_to_visualize, layer, model):
    inputs = [K.learning_phase()] + model.inputs

    _convout1_f = K.function(inputs, [layer.output])
    def convout1_f(X):
        # The [0] is to disable the training phase flag
        return _convout1_f([0] + [X])

    convolutions = convout1_f(img_to_visualize)
    convolutions = np.squeeze(convolutions)
    convolutions=np.moveaxis(convolutions, 2, 0)
    print ('Shape of conv:', convolutions.shape)
    
    filters = convolutions.shape[0]
    print("convs", filters)
    n = int(np.ceil(np.sqrt(filters)))
#    print("sqrted", n)
    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(32,32))
    for i in range(filters):
        ax = fig.add_subplot(n,n,i+1)

        ax.imshow(convolutions[i], cmap='gray')


In [10]:
datagen = CityscapesFlowGenerator(dataset_path, 50)

('-- Cityscapes Previous skip', 0)
('-- Optical flow type', 'dis')
--- flip False
/home/mlyko/data/cityscapes/
Cityscapes: train 2975 files
Cityscapes: val 500 files
Cityscapes: test 1525 files
training samples 50, validating samples 25, test samples 1525


In [18]:
import time
import pylab as pl
from IPython import display

f, arr= plt.subplots(3,1)
f.set_size_inches(15,15)

for imgBatch, labelBatch in datagen.flow('train',1,target_size):
#    imgOld = imgBatch[0]
    imgNew = imgBatch[1][0]
#    print(imgOld.shape)
#    plt.imshow(imgOld)
    #flow = imgBatch[2][0]
#    print(imgNew.shape, imgOld.shape, flow.shape)
    
#    to_vis = np.array([imgOld, imgNew, flow])


    prediction = model.k.predict(imgBatch, batch_size, verbose=1)    
    
    
    colored_class_image = datagen.one_hot_to_bgr(prediction, target_size, datagen.n_classes, datagen.labels)

    predicting_image = cv2.cvtColor( imgNew,  cv2.COLOR_BGR2RGB)
    arr[0].imshow(predicting_image)
    arr[1].imshow(colored_class_image)
    arr[2].imshow(datagen.one_hot_to_bgr(labelBatch, target_size, datagen.n_classes, datagen.labels))

    display.clear_output(wait=True)
    display.display(pl.gcf())
    time.sleep(2.0)
    

    #norm = data_generator.SimpleSegmentationGenerator.normalize(image, (target_height,target_width))

#    to_vis = np.array([norm])
    #to_vis = np.expand_dims(norm, axis=0)
    #print(to_vis.shape)
    
#    to_vis = np.array([imgOld, imgNew])
#    print(imgBatch.shape)
#    layer_to_visualize(np.array([imgOld]), model.k.get_layer('conv2d_27'), model.k)    
 #   layer_to_visualize(np.array([imgOld]), model.k.get_layer('conv2d_19'), model.k)    
    
#    layer_to_visualize(img, model.k.get_layer('warp'), model.k)    

#    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_1'), model.k)    
#    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_4'), model.k)    
    
    
    

ValueError: Floating point image RGB values must be in the 0..1 range.

KeyboardInterrupt: 

ValueError: Floating point image RGB values must be in the 0..1 range.

In [14]:
for imgBatch, labelBatch in datagen.flow('train',1,target_size):
    imgOld = imgBatch[0]
    imgNew = imgBatch[1][0]    
    
    layer_to_visualize(imgBatch[0], model.k.get_layer('conv2d_8'), model.k)    
    
    break

#    layer_to_visualize(np.array([imgOld]), model.k.get_layer('conv2d_27'), model.k)    
 #   layer_to_visualize(np.array([imgOld]), model.k.get_layer('conv2d_19'), model.k)    
    
#    layer_to_visualize(img, model.k.get_layer('warp'), model.k)    

#    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_1'), model.k)    
#    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_4'), model.k)    
    
    
    

InvalidArgumentError: You must feed a value for placeholder tensor 'data_new_1' with dtype float and shape [?,256,512,3]
	 [[Node: data_new_1 = Placeholder[dtype=DT_FLOAT, shape=[?,256,512,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'data_new_1', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_obj, events)
  File "/usr/local/lib

In [17]:
!ls /home/mlyko/data/cityscapes/leftImg8bit/train/

aachen	cologne     erfurt   jena	      strasbourg  ulm
bochum	darmstadt   hamburg  krefeld	      stuttgart   weimar
bremen	dusseldorf  hanover  monchengladbach  tubingen	  zurich


In [29]:
cityscapes_labels.labels

[Label(name='unlabeled', id=0, trainId=0, category='void', categoryId=0, hasInstances=False, ignoreInEval=True, color=(0, 0, 0)),
 Label(name='road', id=7, trainId=1, category='flat', categoryId=1, hasInstances=False, ignoreInEval=False, color=(128, 64, 128)),
 Label(name='sidewalk', id=8, trainId=2, category='flat', categoryId=1, hasInstances=False, ignoreInEval=False, color=(244, 35, 232)),
 Label(name='building', id=11, trainId=3, category='construction', categoryId=2, hasInstances=False, ignoreInEval=False, color=(70, 70, 70)),
 Label(name='wall', id=12, trainId=4, category='construction', categoryId=2, hasInstances=False, ignoreInEval=False, color=(102, 102, 156)),
 Label(name='fence', id=13, trainId=5, category='construction', categoryId=2, hasInstances=False, ignoreInEval=False, color=(190, 153, 153)),
 Label(name='pole', id=17, trainId=6, category='object', categoryId=3, hasInstances=False, ignoreInEval=False, color=(153, 153, 153)),
 Label(name='traffic light', id=19, trainId=